# "Adult" Dataset

In [304]:
# data manipulation
from scipy.io import arff
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [305]:
from sklearn.datasets import fetch_openml

data = fetch_openml(data_id=1590, parser='auto')

# The returned dataset is a Bunch object, similar to a dictionary
X = data['data']
y = data['target']

In [306]:
# Summary vectors creation

default_summary  = []
encoder_summary  = []
value_summary    = []
time_summary     = []
n_models_summary = []
card_9_summary   = []

In [307]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


In [308]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  int64   
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  int64   
 3   education       48842 non-null  category
 4   education-num   48842 non-null  int64   
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capital-gain    48842 non-null  int64   
 11  capital-loss    48842 non-null  int64   
 12  hours-per-week  48842 non-null  int64   
 13  native-country  47985 non-null  category
dtypes: category(8), int64(6)
memory usage: 2.6 MB


In [309]:
X.shape

(48842, 14)

In [310]:
X.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [311]:
X_columns = [col.replace('-','_') for col in X.columns]
X.columns = X_columns

## Variables

We can observe that the variables **education** and **education_num** are completely correlated. We observe that for each value of **education**, there does not exist more than one value of **education_num** and vice versa. Then, we confirm our statement with a chi-squared test to confirm the 'correlation' between the two variables.

In [312]:
X_copy = X.copy()
X_copy['education_num'] = X_copy['education_num'].astype(str)

ncode_forrole = X_copy.groupby('education', as_index=False, observed=True)['education_num'].nunique()
max_rolecode = np.max(ncode_forrole['education_num'])
print(f'Max education_num for each education: {max_rolecode}')

Max education_num for each education: 1


In [313]:
ncode_forrole = X_copy.groupby('education_num', as_index=False, observed=True)['education'].nunique()
max_rolecode = np.max(ncode_forrole['education'])
print(f'Max education for each education_num: {max_rolecode}')

Max education for each education_num: 1


In [314]:
contingency_table = pd.crosstab(X_copy['education'], X_copy['education_num'])

# Realizar la prueba chi-cuadrado
chi2, p_valor, _, _ = chi2_contingency(contingency_table)

print(f"chi-squared value: {chi2}")
print(f"p-value: {p_valor}")

chi-squared value: 732630.0
p-value: 0.0


In [315]:
X = X.drop('education_num', axis=1)

## Study of NA's

In [316]:
pd.DataFrame({
    'Count':X.isna().sum().sort_values(ascending = False),
    'Proportion':(X.isna().sum().sort_values(ascending = False)/len(X))*100
})

,Count,Proportion
occupation,2809,5.751198
workclass,2799,5.730724
native_country,857,1.754637
age,0,0.000000
fnlwgt,0,0.000000
education,0,0.000000
marital_status,0,0.000000
relationship,0,0.000000
race,0,0.000000
sex,0,0.000000


As can be seen, there are three features with np.nan values.

## Type of Variables

In [317]:
num_cols = X.select_dtypes(exclude=['object','category']).columns
binary_cols = [col for col in X.columns if X[col].isin([0, 1, 0.0, 1.0]).all()]
num_cols = [num_col for num_col in num_cols if num_col not in binary_cols]

print(f'Numeric columns: {len(num_cols)}')
print(f'Binary columns: {len(binary_cols)}')

Numeric columns: 5
Binary columns: 0


In [318]:
cat_columns = X.select_dtypes(include=['category']).columns
print(f'Category columns: {len(cat_columns)}')

Category columns: 8


General review of the values of all variables.

In [319]:
X[cat_columns].apply(lambda col: col.nunique()).sort_values(ascending=False)

native_country    41
education         16
occupation        14
workclass          8
marital_status     7
relationship       6
race               5
sex                2
dtype: int64

## Value counts of the variables with more cardinality

#### education

In [320]:
X.education.value_counts()[0:50]

education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

#### occupation

In [321]:
X.occupation.value_counts()[0:10]

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Name: count, dtype: int64

#### workclass

In [322]:
X.workclass.value_counts()[0:10]

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

## Response variable distribution

In [323]:
y = y.replace({'<=50K': 0, '>50K': 1})
y.head()

0    0
1    0
2    1
3    1
4    0
Name: class, dtype: category
Categories (2, int64): [0, 1]

In [324]:
y.value_counts()

class
0    37155
1    11687
Name: count, dtype: int64

In [325]:
y.value_counts(normalize=True)

class
0    0.760718
1    0.239282
Name: proportion, dtype: float64

## Train-Test Split

In [326]:
from sklearn.model_selection import train_test_split

In [327]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size = 0.33, 
                                                    random_state = 42,
                                                    stratify = y)

## Pipelines 

In [328]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import balanced_accuracy_score

import scipy.stats
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

import time

In [329]:
num_cols = X_train.select_dtypes(include=['number']).columns.to_list()
cat_cols = X_train.select_dtypes(include=['category']).columns.to_list()

cat_cols_less9 = [col for col in cat_cols if X[col].nunique() <= 9]
cat_cols_more9 = [col for col in cat_cols if X[col].nunique() > 9]

In [330]:
# Define the HistGradientBoostingClassifier models
hgb_default = HistGradientBoostingClassifier(max_iter=1000, random_state=1234,
                                             early_stopping=True,
                                             scoring='balanced_accuracy',
                                             validation_fraction=0.1,
                                             n_iter_no_change=5,
                                             class_weight='balanced')

hgb_default9 = HistGradientBoostingClassifier(max_iter=1000, random_state=1234,
                                              early_stopping=True,
                                              scoring='balanced_accuracy',
                                              validation_fraction=0.1,
                                              n_iter_no_change=5,
                                              class_weight='balanced')

# Define the hyperparameter search space
param_distributions = {
    'model__max_iter': scipy.stats.randint(10,1000),
    'model__learning_rate': scipy.stats.uniform(0.01, 0.3),
    'model__min_samples_leaf': scipy.stats.randint(5, 15),
}

# Create a StratifiedKFold cross-validation instance
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

### One Hot Encoding + HistGradientBoosting

#### Preprocessing

In [331]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "median"))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot", cat_pipeline, cat_cols)],
    sparse_threshold=0
)

#### Create a HistGradientBoostingClassifier model with default parameters and early stopping

In [332]:
ohe_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                     ('model', hgb_default)])

In [333]:
tic = time.time()

ohe_hgb_default = ohe_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
ohe_hgb_default_time_taken = toc-tic

In [334]:
# Display pipeline
print("Time taken: ", ohe_hgb_default_time_taken)
ohe_hgb_default

Time taken:  0.9966869354248047


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('one_hot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [335]:
# # Calculate balanced accuracy for the model with default parameters
y_ohe_hgb_default_pred = ohe_hgb_default.predict(X_test)
ohe_hgb_default_accuracy = balanced_accuracy_score(y_test, y_ohe_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {ohe_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OneHotEncoding")
value_summary.append(ohe_hgb_default_accuracy)
time_summary.append(ohe_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8346286309244421


#### Create a HistGradientBoostingClassifier model for tuning

In [336]:
ohe_hgb_tune = RandomizedSearchCV(estimator = ohe_hgb_default_pipeline, 
                                  param_distributions = param_distributions, 
                                  n_iter = 200,
                                  cv = stratified_kfold,
                                  scoring = 'balanced_accuracy', 
                                  random_state = 1234,
                                  n_jobs = -1)

In [337]:
tic = time.time()

ohe_hgb_tune = ohe_hgb_tune.fit(X_train, y_train)

toc = time.time()
ohe_hgb_tune_time_taken = toc-tic

In [338]:
# Display pipeline
print("Time taken: ", ohe_hgb_tune_time_taken)
ohe_hgb_tune

Time taken:  109.91694283485413


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('one_hot',
                                                                               Pipeline(steps=[('imputer',
                                                                                                Si...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [339]:
# Get the best parameters
ohe_hgb_tune_best_params = ohe_hgb_tune.best_params_
print(f'Best parameters: {ohe_hgb_tune_best_params}')

# Predict using the model with the best parameters
y_ohe_hgb_tune_pred = ohe_hgb_tune.predict(X_test)
ohe_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_ohe_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {ohe_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("OneHotEncoding")
value_summary.append(ohe_hgb_tune_accuracy)
time_summary.append(ohe_hgb_tune_time_taken)
n_models_summary.append(ohe_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.2909671002703995, 'model__max_iter': 625, 'model__min_samples_leaf': 11}
Balanced accuracy with best parameters: 0.8403116009525514


### Count Encoder + HistGradientBoosting


In [340]:
from category_encoders.count import CountEncoder

#### Preprocessing

In [341]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", CountEncoder())
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("count_encoder", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("count_encoder", cat_pipeline_more9, cat_cols_more9)],
    sparse_threshold=0
)

#### Create a HistGradientBoostingClassifier model with default parameters and early stopping

In [342]:
count_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                       ('model', hgb_default)])
count_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                        ('model', hgb_default9)])

In [343]:
tic = time.time()

count_hgb_default = count_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
count_hgb_default_time_taken = toc-tic

In [344]:
# Display pipeline
print("Time taken: ", count_hgb_default_time_taken)
count_hgb_default

Time taken:  1.6224658489227295


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('count_encoder',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [345]:
tic = time.time()

count_hgb_default9 = count_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
count_hgb_default9_time_taken = toc-tic

In [346]:
# Display pipeline
print("Time taken: ", count_hgb_default9_time_taken)
count_hgb_default9

Time taken:  1.8167634010314941


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('one_hot_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handl...
                                                 ('count_encoder',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   CountEncoder(combine_min_nan_groups=True))]),
                                                  ['education', 'occupation',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [347]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_pred = count_hgb_default.predict(X_test)
count_hgb_default_accuracy = balanced_accuracy_score(y_test, y_count_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {count_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("CountEncoding")
value_summary.append(count_hgb_default_accuracy)
time_summary.append(count_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8408885011309946


In [348]:
# Calculate balanced accuracy for the model with default parameters
y_count_hgb_default_pred9 = count_hgb_default9.predict(X_test)
count_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_count_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {count_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CountEncoding")
value_summary.append(count_hgb_default_accuracy9)
time_summary.append(count_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8410602263105685


#### Create a HistGradientBoostingClassifier model for tuning

In [349]:
count_hgb_tune = RandomizedSearchCV(estimator = count_hgb_default_pipeline, 
                                    param_distributions = param_distributions, 
                                    n_iter = 200,
                                    cv = stratified_kfold,
                                    scoring = 'balanced_accuracy', 
                                    random_state = 1234,
                                    n_jobs = -1)

count_hgb_tune9 = RandomizedSearchCV(estimator = count_hgb_default_pipeline9, 
                                     param_distributions = param_distributions, 
                                     n_iter = 200,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

In [350]:
tic = time.time()

count_hgb_tune = count_hgb_tune.fit(X_train, y_train)

toc = time.time()
count_hgb_tune_time_taken = toc-tic

In [351]:
# Display pipeline
print("Time taken: ", count_hgb_tune_time_taken)
count_hgb_tune

Time taken:  76.73618268966675


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('count_encoder',
                                                                               Pipeline(steps=[('imput...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [352]:
tic = time.time()

count_hgb_tune9 = count_hgb_tune9.fit(X_train, y_train)

toc = time.time()
count_hgb_tune9_time_taken = toc-tic

In [353]:
# Display pipeline
print("Time taken: ", count_hgb_tune9_time_taken)
count_hgb_tune9

Time taken:  82.96799969673157


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('one_hot_encoding',
                                                                               Pipeline(steps=[('im...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [354]:
# Predict using the model with the best parameters
y_count_hgb_tune_pred = count_hgb_tune.predict(X_test)

# Get the best parameters
count_hgb_tune_best_params = count_hgb_tune.best_params_
print(f'Best parameters: {count_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
count_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_count_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {count_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("CountEncoding")
value_summary.append(count_hgb_tune_accuracy)
time_summary.append(count_hgb_tune_time_taken)
n_models_summary.append(count_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.24479077305647257, 'model__max_iter': 154, 'model__min_samples_leaf': 3}
Balanced accuracy with best parameters: 0.8393620691875483


In [355]:
# Predict using the model with the best parameters
y_count_hgb_tune_pred9 = count_hgb_tune9.predict(X_test)

# Get the best parameters
count_hgb_tune_best_params9 = count_hgb_tune9.best_params_
print(f'Best parameters: {count_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
count_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_count_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {count_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CountEncoding")
value_summary.append(count_hgb_tune_accuracy9)
time_summary.append(count_hgb_tune9_time_taken)
n_models_summary.append(count_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.23285376094301305, 'model__max_iter': 470, 'model__min_samples_leaf': 1}
Balanced accuracy with best parameters: 0.8383962657163906


### Ordinal Encoding + HistGradientBoosting


In [356]:
from sklearn.preprocessing import OrdinalEncoder

#### Preprocessing

In [357]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OrdinalEncoder(dtype = int,
                               handle_unknown = 'use_encoded_value',
                               unknown_value = 99999,
                               encoded_missing_value = 99999))
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("ordinal_encoder", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("ordinalencoder", cat_pipeline_more9, cat_cols_more9)
                   ],
    sparse_threshold=0
)

#### Create a HistGradientBoostingClassifier model with default parameters and early stopping

In [358]:
ordinal_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', hgb_default)])

ordinal_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                          ('model', hgb_default9)])

In [359]:
tic = time.time()

ordinal_hgb_default = ordinal_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default_time_taken = toc-tic

In [360]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default_time_taken)
ordinal_hgb_default

Time taken:  0.42326784133911133


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('ordinal_encoder',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(dtype=<class 'int'>,
                                                                                  encoded_missing_value=99999,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999))]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [361]:
tic = time.time()

ordinal_hgb_default9 = ordinal_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_default9_time_taken = toc-tic

In [362]:
# Display pipeline
print("Time taken: ", ordinal_hgb_default9_time_taken)
ordinal_hgb_default9

Time taken:  1.2102892398834229


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('one_hot_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handl...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(dtype=<class 'int'>,
                                                                                  encoded_missing_value=99999,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999))]),
                                                  ['education', 'occupation',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [363]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_pred = ordinal_hgb_default.predict(X_test)
ordinal_hgb_default_accuracy = balanced_accuracy_score(y_test, y_ordinal_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("OrdinalEncoder")
value_summary.append(ordinal_hgb_default_accuracy)
time_summary.append(ordinal_hgb_default_time_taken)
n_models_summary.append(1)


Balanced accuracy with default parameters: 0.8230541191025875


In [364]:
# Calculate balanced accuracy for the model with default parameters
y_ordinal_hgb_default_pred9 = ordinal_hgb_default9.predict(X_test)
ordinal_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_ordinal_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {ordinal_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("OrdinalEncoder")
value_summary.append(ordinal_hgb_default_accuracy9)
time_summary.append(ordinal_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8354558235653933


#### Create a HistGradientBoostingClassifier model for tuning

In [365]:
ordinal_hgb_tune = RandomizedSearchCV(estimator = ordinal_hgb_default_pipeline, 
                                      param_distributions = param_distributions, 
                                      n_iter = 200,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                      n_jobs = -1)

ordinal_hgb_tune9 = RandomizedSearchCV(estimator = ordinal_hgb_default_pipeline9, 
                                       param_distributions = param_distributions, 
                                       n_iter = 200,
                                       cv = stratified_kfold,
                                       scoring = 'balanced_accuracy', 
                                       random_state = 1234,
                                       n_jobs = -1)

In [366]:
tic = time.time()

ordinal_hgb_tune = ordinal_hgb_tune.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_tune_time_taken = toc-tic

In [367]:
# Display pipeline
print("Time taken: ", ordinal_hgb_tune_time_taken)
ordinal_hgb_tune

Time taken:  73.55292010307312


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('ordinal_encoder',
                                                                               Pipeline(steps=[('imp...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [368]:
tic = time.time()

ordinal_hgb_tune9 = ordinal_hgb_tune9.fit(X_train, y_train)

toc = time.time()
ordinal_hgb_tune9_time_taken = toc-tic

In [369]:
# Display pipeline
print("Time taken: ", ordinal_hgb_tune9_time_taken)
ordinal_hgb_tune9

Time taken:  64.16526293754578


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('one_hot_encoding',
                                                                               Pipeline(steps=[('im...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [370]:
# Predict using the model with the best parameters
y_ordinal_hgb_tune_pred = ordinal_hgb_tune.predict(X_test)

# Get the best parameters
ordinal_hgb_tune_best_params = ordinal_hgb_tune.best_params_
print(f'Best parameters: {ordinal_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
ordinal_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_ordinal_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {ordinal_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("OrdinalEncoder")
value_summary.append(ordinal_hgb_tune_accuracy)
time_summary.append(ordinal_hgb_tune_time_taken)
n_models_summary.append(ordinal_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.2600760702063973, 'model__max_iter': 570, 'model__min_samples_leaf': 14}
Balanced accuracy with best parameters: 0.8399704026229102


In [371]:
# Predict using the model with the best parameters
y_ordinal_hgb_tune_pred9 = ordinal_hgb_tune9.predict(X_test)

# Get the best parameters
ordinal_hgb_tune_best_params9 = ordinal_hgb_tune9.best_params_
print(f'Best parameters: {ordinal_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
ordinal_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_ordinal_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {ordinal_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("OrdinalEncoder")
value_summary.append(ordinal_hgb_tune_accuracy9)
time_summary.append(ordinal_hgb_tune9_time_taken)
n_models_summary.append(ordinal_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.30669449723883385, 'model__max_iter': 331, 'model__min_samples_leaf': 16}
Balanced accuracy with best parameters: 0.8392818208121655


### Native HistGradientBoosting support for categorical variables


In [372]:
from sklearn.preprocessing import OrdinalEncoder

#### Preprocessing

In [373]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OrdinalEncoder(dtype = int,
                               handle_unknown = 'use_encoded_value',
                               unknown_value = 99999,
                               encoded_missing_value = 99999,
                               max_categories = 254))
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("ordinal_encoding", cat_pipeline_more9, cat_cols_more9)
                    ],
    sparse_threshold=0
)

In [374]:
category_features_for_nativesupport = [False]*len(num_cols) + [True]*len(cat_cols)

X_train_check = preprop_pipeline9.fit_transform(X_train)
category_features_for_nativesupport_9 = [False]*(X_train_check.shape[1]-len(cat_cols_more9)) + [True]*len(cat_cols_more9)

In [375]:
hgb_default_categories_support = HistGradientBoostingClassifier(max_iter=1000, random_state=1234,
                                                                early_stopping=True,
                                                                scoring='balanced_accuracy',
                                                                validation_fraction=0.1,
                                                                n_iter_no_change=5,
                                                                class_weight='balanced',
                                                                categorical_features=category_features_for_nativesupport)

hgb_default_categories_support9 = HistGradientBoostingClassifier(max_iter=1000, random_state=1234,
                                                                 early_stopping=True,
                                                                 scoring='balanced_accuracy',
                                                                 validation_fraction=0.1,
                                                                 n_iter_no_change=5,
                                                                 class_weight='balanced',
                                                                 categorical_features=category_features_for_nativesupport_9)

#### Create a HistGradientBoostingClassifier model with default parameters and early stopping

In [376]:
catsup_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                        ('model', hgb_default_categories_support)])

catsup_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                        ('model', hgb_default_categories_support9)])

In [377]:
tic = time.time()

catsup_hgb_default = catsup_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default_time_taken = toc-tic

In [378]:
# Display pipeline
print("Time taken: ", catsup_hgb_default_time_taken)
catsup_hgb_default

Time taken:  0.3302884101867676


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(dtype=<class 'int'>,
                                                                                  encoded_mis...
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True,
                                                                      True],
                                                class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [379]:
tic = time.time()

catsup_hgb_default9 = catsup_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
catsup_hgb_default9_time_taken = toc-tic

In [380]:
# Display pipeline
print("Time taken: ", catsup_hgb_default9_time_taken)
catsup_hgb_default9

Time taken:  0.7437996864318848


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('one_hot_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handl...
                 HistGradientBoostingClassifier(categorical_features=[False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      False,
                                                                      True,
                                                                      True, ...],
                                                class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [381]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_pred = catsup_hgb_default.predict(X_test)
catsup_hgb_default_accuracy = balanced_accuracy_score(y_test, y_catsup_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("HGB_NativeSupport")
value_summary.append(catsup_hgb_default_accuracy)
time_summary.append(catsup_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8259885769873838


In [382]:
# Calculate balanced accuracy for the model with default parameters
y_catsup_hgb_default_pred9 = catsup_hgb_default9.predict(X_test)
catsup_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_catsup_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {catsup_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("HGB_NativeSupport")
value_summary.append(catsup_hgb_default_accuracy9)
time_summary.append(catsup_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8352137673985931


#### Create a HistGradientBoostingClassifier model for tuning

In [383]:
catsup_hgb_tune = RandomizedSearchCV(estimator = catsup_hgb_default_pipeline, 
                                     param_distributions = param_distributions, 
                                     n_iter = 200,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

catsup_hgb_tune9 = RandomizedSearchCV(estimator = catsup_hgb_default_pipeline9, 
                                      param_distributions = param_distributions, 
                                      n_iter = 200,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                     n_jobs = -1)

In [384]:
tic = time.time() 

catsup_hgb_tune = catsup_hgb_tune.fit(X_train, y_train)

toc = time.time()
catsup_hgb_tune_time_taken = toc-tic

In [385]:
# Display pipeline
print("Time taken: ", catsup_hgb_tune_time_taken)
catsup_hgb_tune

Time taken:  63.346805810928345


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('imputer',
                                                                                                Simple...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [386]:
tic = time.time() 

catsup_hgb_tune9 = catsup_hgb_tune9.fit(X_train, y_train)

toc = time.time()
catsup_hgb_tune9_time_taken = toc-tic

In [387]:
# Display pipeline
print("Time taken: ", catsup_hgb_tune9_time_taken)
catsup_hgb_tune9

Time taken:  71.1726655960083


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('one_hot_encoding',
                                                                               Pipeline(steps=[('im...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [388]:
# Predict using the model with the best parameters
y_catsup_hgb_tune_pred = catsup_hgb_tune.predict(X_test)

# Get the best parameters
catsup_hgb_tune_best_params = catsup_hgb_tune.best_params_
print(f'Best parameters: {catsup_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
catsup_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_catsup_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {catsup_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("HGB_NativeSupport")
value_summary.append(catsup_hgb_tune_accuracy)
time_summary.append(catsup_hgb_tune_time_taken)
n_models_summary.append(catsup_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.28591777801530216, 'model__max_iter': 440, 'model__min_samples_leaf': 14}
Balanced accuracy with best parameters: 0.8396252711713135


In [389]:
# Predict using the model with the best parameters
y_catsup_hgb_tune_pred9 = catsup_hgb_tune9.predict(X_test)

# Get the best parameters
catsup_hgb_tune_best_params9 = catsup_hgb_tune9.best_params_
print(f'Best parameters: {catsup_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
catsup_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_catsup_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {catsup_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("HGB_NativeSupport")
value_summary.append(catsup_hgb_tune_accuracy9)
time_summary.append(catsup_hgb_tune9_time_taken)
n_models_summary.append(catsup_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.30486096527393985, 'model__max_iter': 63, 'model__min_samples_leaf': 9}
Balanced accuracy with best parameters: 0.8408048905707144


### Target Encoder

In [390]:
from sklearn.preprocessing import TargetEncoder

#### Preprocessing

In [391]:
cat_pipeline_more9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", TargetEncoder())
])

cat_pipeline_less9 = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("encoder", OneHotEncoder(drop = "first", handle_unknown = "ignore"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat", cat_pipeline_more9, cat_cols)],
    sparse_threshold=0
)

preprop_pipeline9 = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("one_hot_encoding", cat_pipeline_less9, cat_cols_less9),
                    ("target_encoding", cat_pipeline_more9, cat_cols_more9)
                    ],
    sparse_threshold=0
)

#### Create a HistGradientBoostingClassifier model with default parameters and early stopping

In [392]:
target_hgb_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                       ('model', hgb_default)])

target_hgb_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline9),
                                       ('model', hgb_default9)])

In [393]:
tic = time.time()

target_hgb_default = target_hgb_default_pipeline.fit(X_train, y_train)

toc = time.time()
target_hgb_default_time_taken = toc-tic

In [394]:
# Display pipeline
print("Time taken: ", target_hgb_default_time_taken)
target_hgb_default

Time taken:  0.6756236553192139


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [395]:
tic = time.time()

target_hgb_default9 = target_hgb_default_pipeline9.fit(X_train, y_train)

toc = time.time()
target_hgb_default9_time_taken = toc-tic

In [396]:
# Display pipeline
print("Time taken: ", target_hgb_default9_time_taken)
target_hgb_default9

Time taken:  2.368952512741089


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('one_hot_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handl...
                                                   'relationship', 'race',
                                                   'sex']),
                                                 ('target_encoding',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['education', 'occupation',
                                                   'native_country'])])),
                ('model',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=5,
                                                random_state=1234,
                                                scoring='balanced_accuracy'))])

In [397]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_pred = target_hgb_default.predict(X_test)
target_hgb_default_accuracy = balanced_accuracy_score(y_test, y_target_hgb_default_pred)
print(f'Balanced accuracy with default parameters: {target_hgb_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("TargetEncoder")
value_summary.append(target_hgb_default_accuracy)
time_summary.append(target_hgb_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.834657442100057


In [398]:
# Calculate balanced accuracy for the model with default parameters
y_target_hgb_default_pred9 = target_hgb_default9.predict(X_test)
target_hgb_default_accuracy9 = balanced_accuracy_score(y_test, y_target_hgb_default_pred9)
print(f'Balanced accuracy with default parameters: {target_hgb_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("TargetEncoder")
value_summary.append(target_hgb_default_accuracy9)
time_summary.append(target_hgb_default9_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8410658405249728


#### Create a HistGradientBoostingClassifier model for tuning

In [399]:
target_hgb_tune = RandomizedSearchCV(estimator = target_hgb_default_pipeline, 
                                     param_distributions = param_distributions, 
                                     n_iter = 200,
                                     cv = stratified_kfold,
                                     scoring = 'balanced_accuracy', 
                                     random_state = 1234,
                                     n_jobs = -1)

target_hgb_tune9 = RandomizedSearchCV(estimator = target_hgb_default_pipeline9, 
                                      param_distributions = param_distributions, 
                                      n_iter = 200,
                                      cv = stratified_kfold,
                                      scoring = 'balanced_accuracy', 
                                      random_state = 1234,
                                      n_jobs = -1)

In [400]:
tic = time.time() 

target_hgb_tune = target_hgb_tune.fit(X_train, y_train)

toc = time.time()
target_hgb_tune_time_taken = toc-tic

In [401]:
# Display pipeline
print("Time taken: ", target_hgb_tune_time_taken)
target_hgb_tune

Time taken:  60.887014389038086


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('imputer',
                                                                                                Simple...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [402]:
tic = time.time() 

target_hgb_tune9 = target_hgb_tune9.fit(X_train, y_train)

toc = time.time()
target_hgb_tune9_time_taken = toc-tic

In [403]:
# Display pipeline
print("Time taken: ", target_hgb_tune9_time_taken)
target_hgb_tune9

Time taken:  64.24427437782288


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('one_hot_encoding',
                                                                               Pipeline(steps=[('im...
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC06CCDC00>,
                                        'model__max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC05FAFA30>,
                                        'model__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC06CCE1A0>},
                   random_state=1234, scoring='balanced_accuracy')

In [404]:
# Predict using the model with the best parameters
y_target_hgb_tune_pred = target_hgb_tune.predict(X_test)

# Get the best parameters
target_hgb_tune_best_params = target_hgb_tune.best_params_
print(f'Best parameters: {target_hgb_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
target_hgb_tune_accuracy = balanced_accuracy_score(y_test, y_target_hgb_tune_pred)
print(f'Balanced accuracy with best parameters: {target_hgb_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("TargetEncoder")
value_summary.append(target_hgb_tune_accuracy)
time_summary.append(target_hgb_tune_time_taken)
n_models_summary.append(target_hgb_tune.n_iter)

Best parameters: {'model__learning_rate': 0.2377474245702175, 'model__max_iter': 784, 'model__min_samples_leaf': 18}
Balanced accuracy with best parameters: 0.8401481120686853


In [405]:
# Predict using the model with the best parameters
y_target_hgb_tune_pred9 = target_hgb_tune9.predict(X_test)

# Get the best parameters
target_hgb_tune_best_params9 = target_hgb_tune9.best_params_
print(f'Best parameters: {target_hgb_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
target_hgb_tune_accuracy9 = balanced_accuracy_score(y_test, y_target_hgb_tune_pred9)
print(f'Balanced accuracy with best parameters: {target_hgb_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("TargetEncoder")
value_summary.append(target_hgb_tune_accuracy9)
time_summary.append(target_hgb_tune9_time_taken)
n_models_summary.append(target_hgb_tune9.n_iter)

Best parameters: {'model__learning_rate': 0.27063407326327066, 'model__max_iter': 517, 'model__min_samples_leaf': 13}
Balanced accuracy with best parameters: 0.8412886328525175


### CatBoost

In [406]:
from catboost import CatBoostClassifier

#### Preprocessing

In [407]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent"))
])

preprop_pipeline = ColumnTransformer(
    transformers = [("num", num_pipeline, num_cols),
                    ("cat_less9", cat_pipeline, cat_cols_less9),
                    ("cat_more9", cat_pipeline, cat_cols_more9)],
    sparse_threshold=0
)

In [408]:
category_features_for_catboostsupport = [index for index in range(len(num_cols), len(num_cols) + len(cat_cols))]
category_features_for_catboostsupport9 = [index for index in range(len(num_cols) + len(cat_cols_less9), len(num_cols) + len(cat_cols))]
print(category_features_for_catboostsupport)
print(category_features_for_catboostsupport9)

[5, 6, 7, 8, 9, 10, 11, 12]
[10, 11, 12]


Catboost allows to give a maximum value of unique categories for which a variable is encoded or not by One-Hot-Encoder.

In [409]:
# Create catboost models
catboost_default_raw = CatBoostClassifier(eval_metric = 'BalancedAccuracy',
                                          loss_function = 'Logloss',
                                          auto_class_weights = 'Balanced',
                                          od_type='Iter',
                                          one_hot_max_size = 9,
                                          random_seed = 1234,
                                          cat_features=category_features_for_catboostsupport,
                                          verbose = False)

catboost_default9_raw = CatBoostClassifier(eval_metric = 'BalancedAccuracy',
                                           loss_function = 'Logloss',
                                           auto_class_weights = 'Balanced',
                                           od_type='Iter',
                                           one_hot_max_size = 9,
                                           random_seed = 1234,
                                           cat_features=category_features_for_catboostsupport9,
                                           verbose = False)


# catboost_default_raw.set_params(cat_features=category_features_for_catboostsupport)
# catboost_default9_raw.set_params(cat_features=category_features_for_catboostsupport9)


# Default CatBoostClassifier Pipeline
catboost_default_pipeline = Pipeline([('preprocessing', preprop_pipeline),
                                        ('model', catboost_default_raw)])

catboost_default_pipeline9 = Pipeline([('preprocessing', preprop_pipeline),
                                         ('model', catboost_default9_raw)])

# Define the hyperparameter search space
catboost_param_distributions = {
    'model__iterations': scipy.stats.randint(10, 1000),
    'model__learning_rate': scipy.stats.uniform(0.01, 0.3),
    'model__min_data_in_leaf': scipy.stats.randint(5,15),
}

# Create a StratifiedKFold cross-validation instance
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

catboost_tune_raw = RandomizedSearchCV(estimator = catboost_default_pipeline, 
                                   param_distributions = catboost_param_distributions, 
                                   n_iter = 5,
                                   cv = stratified_kfold,
                                   scoring = 'balanced_accuracy', 
                                   random_state = 1234,
                                   n_jobs = -1)


catboost_tune9_raw = RandomizedSearchCV(estimator = catboost_default_pipeline, 
                                        param_distributions = catboost_param_distributions, 
                                        n_iter = 5,
                                        cv = stratified_kfold,
                                        scoring = 'balanced_accuracy', 
                                        random_state = 1234,
                                        n_jobs = -1)

In [410]:
tic = time.time()

catboost_default = catboost_default_pipeline.fit(X_train, y_train)

toc = time.time()
catboost_default_time_taken = toc-tic

In [411]:
# Display pipeline
print("Time taken: ", catboost_default_time_taken)
catboost_default

Time taken:  57.84385371208191


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('cat_less9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['workclass',
                                                   'marital_status',
                                                   'relationship', 'race',
                                                   'sex']),
                                                 ('cat_more9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['education', 'occupation',
                                                   'native_country'])])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001DC7448B160>)])

In [412]:
tic = time.time()

catboost_default9 = catboost_default_pipeline.fit(X_train, y_train)

toc = time.time()
catboost_default_time_taken9 = toc-tic

In [413]:
# Display pipeline
print("Time taken: ", catboost_default_time_taken9)
catboost_default9

Time taken:  62.930856227874756


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'fnlwgt',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_per_week']),
                                                 ('cat_less9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['workclass',
                                                   'marital_status',
                                                   'relationship', 'race',
                                                   'sex']),
                                                 ('cat_more9',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['education', 'occupation',
                                                   'native_country'])])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001DC7448B160>)])

In [414]:
# Calculate balanced accuracy for the model with default parameters
y_catboost_default_pred = catboost_default.predict(X_test)
catboost_default_accuracy = balanced_accuracy_score(y_test, y_catboost_default_pred)
print(f'Balanced accuracy with default parameters: {catboost_default_accuracy}')

# Save results
default_summary.append("Default")
card_9_summary.append("AllVariables")
encoder_summary.append("CatboostNativeSupport")
value_summary.append(catboost_default_accuracy)
time_summary.append(catboost_default_time_taken)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8404315188805607


In [415]:
# Calculate balanced accuracy for the model with default parameters
y_catboost_default_pred9 = catboost_default9.predict(X_test)
catboost_default_accuracy9 = balanced_accuracy_score(y_test, y_catboost_default_pred9)
print(f'Balanced accuracy with default parameters: {catboost_default_accuracy9}')

# Save results
default_summary.append("Default")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CatboostNativeSupport")
value_summary.append(catboost_default_accuracy9)
time_summary.append(catboost_default_time_taken9)
n_models_summary.append(1)

Balanced accuracy with default parameters: 0.8404315188805607


In [416]:
tic = time.time()

catboost_tune = catboost_tune_raw.fit(X_train, y_train)

toc = time.time()
catboost_tune_time_taken = toc-tic

In [417]:
# Display pipeline
print("Time taken: ", catboost_tune_time_taken)
catboost_tune

Time taken:  118.77689719200134


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('cat_less9',
                                                                               Pipeline(steps=[('imputer',...
                   param_distributions={'model__iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC6DD01690>,
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC6CD4F0A0>,
                                        'model__min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC6E7F8CA0>},
                   random_state=1234, scoring='balanced_accuracy')

In [418]:
tic = time.time()

catboost_tune9 = catboost_tune9_raw.fit(X_train, y_train)

toc = time.time()
catboost_tune_time_taken9 = toc-tic

In [419]:
# Display pipeline
print("Time taken: ", catboost_tune_time_taken9)
catboost_tune9

Time taken:  113.67401695251465


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1234, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(sparse_threshold=0,
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'capital_gain',
                                                                                'capital_loss',
                                                                                'hours_per_week']),
                                                                              ('cat_less9',
                                                                               Pipeline(steps=[('imputer',...
                   param_distributions={'model__iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC6DD01690>,
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001DC6CD4F0A0>,
                                        'model__min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001DC6E7F8CA0>},
                   random_state=1234, scoring='balanced_accuracy')

In [420]:
# Predict using the model with the best parameters
y_catboost_tune_pred = catboost_tune.predict(X_test)

# Get the best parameters
catboost_tune_best_params = catboost_tune.best_params_
print(f'Best parameters: {catboost_tune_best_params}')

# Calculate balanced accuracy for the model with the best parameters
catboost_tune_accuracy = balanced_accuracy_score(y_test, y_catboost_tune_pred)
print(f'Balanced accuracy with best parameters: {catboost_tune_accuracy}')

# Save results
default_summary.append("Tune")
card_9_summary.append("AllVariables")
encoder_summary.append("CatboostNativeSupport")
value_summary.append(catboost_tune_accuracy)
time_summary.append(catboost_tune_time_taken)
n_models_summary.append(catboost_tune.n_iter)

Best parameters: {'model__iterations': 168, 'model__learning_rate': 0.29744180610511156, 'model__min_data_in_leaf': 17}
Balanced accuracy with best parameters: 0.838515242655545


In [421]:
# Predict using the model with the best parameters
y_catboost_tune_pred9 = catboost_tune9.predict(X_test)

# Get the best parameters
catboost_tune_best_params9 = catboost_tune9.best_params_
print(f'Best parameters: {catboost_tune_best_params9}')

# Calculate balanced accuracy for the model with the best parameters
catboost_tune_accuracy9 = balanced_accuracy_score(y_test, y_catboost_tune_pred9)
print(f'Balanced accuracy with best parameters: {catboost_tune_accuracy9}')

# Save results
default_summary.append("Tune")
card_9_summary.append("OnlyVariablesWithCard>9")
encoder_summary.append("CatboostNativeSupport")
value_summary.append(catboost_tune_accuracy9)
time_summary.append(catboost_tune_time_taken9)
n_models_summary.append(catboost_tune9.n_iter)

Best parameters: {'model__iterations': 168, 'model__learning_rate': 0.29744180610511156, 'model__min_data_in_leaf': 17}
Balanced accuracy with best parameters: 0.838515242655545


### Results Summary

In [423]:
results_summary = pd.DataFrame({"Dataset":"adult",
                                "Variables":card_9_summary,
                                "Default/Tune":default_summary,
                                "Encoder":encoder_summary,
                                "Metric":"BalancedAccuracy",
                                "Value":value_summary,
                                "Time":time_summary,
                                "Iterations":n_models_summary})
results_summary["mean_Time"] = (results_summary["Time"] / results_summary["Iterations"])
results_summary

,Dataset,Variables,Default/Tune,Encoder,Metric,Value,Time,Iterations,mean_Time
0,adult,AllVariables,Default,OneHotEncoding,BalancedAccuracy,0.834629,0.996687,1,0.996687
1,adult,AllVariables,Tune,OneHotEncoding,BalancedAccuracy,0.840312,109.916943,200,0.549585
2,adult,AllVariables,Default,CountEncoding,BalancedAccuracy,0.840889,1.622466,1,1.622466
3,adult,OnlyVariablesWithCard>9,Default,CountEncoding,BalancedAccuracy,0.841060,1.816763,1,1.816763
4,adult,AllVariables,Tune,CountEncoding,BalancedAccuracy,0.839362,76.736183,200,0.383681
5,adult,OnlyVariablesWithCard>9,Tune,CountEncoding,BalancedAccuracy,0.838396,82.968000,200,0.414840
6,adult,AllVariables,Default,OrdinalEncoder,BalancedAccuracy,0.823054,0.423268,1,0.423268
7,adult,OnlyVariablesWithCard>9,Default,OrdinalEncoder,BalancedAccuracy,0.835456,1.210289,1,1.210289
8,adult,AllVariables,Tune,OrdinalEncoder,BalancedAccuracy,0.839970,73.552920,200,0.367765
9,adult,OnlyVariablesWithCard>9,Tune,OrdinalEncoder,BalancedAccuracy,0.839282,64.165263,200,0.320826


In [ ]:
results_summary.to_csv("adult_results.csv")